In [4]:
#!pip install awswrangler

In [5]:
import awswrangler as wr
import pandas as pd

In [6]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'
origin_name = 'AwsDataCatalog'
database_name= 'analytics'
table_name = 'daily_check'

In [7]:
import sagemaker
sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


'arn:aws:iam::283731589572:role/service-role/SageMaker-ExecutionRole-20240102T104128'

In [8]:
# Read data from Athena
# df_daily_check = wr.athena.read_sql_query(
#     f'SELECT * FROM {origin_name}.{database_name}.{table_name};',
#     database=database_name
# )


In [9]:
df_daily_check = pd.read_excel("daily_check.xlsx")

In [10]:
# Convertir todos los nombres de columna a minúsculas
df_daily_check.columns = df_daily_check.columns.str.lower()

In [11]:
df_test = pd.read_csv("df_train_prueba2.csv")

In [12]:
# Me traigo solamente las columnas de payer_country y la del encoder para poder cruzar los datos
df_translate = pd.read_csv("df_final_21_10_2023.csv")[['payer_country','payer_country_encoder']]

In [13]:
# Me quedo con una tabla simple, con ocurrencias unicas
df_translate = df_translate.drop_duplicates(subset='payer_country_encoder')

In [14]:
df_translate.to_excel('translate.xlsx')

In [15]:
# Al df_test le agrego la correspondencia payer_country_encoder <> payer_country
df_final = pd.merge(df_test, df_translate, on=['payer_country_encoder'])

In [16]:
#Separo el nombre del pais
df_final['country'] = df_final['payer_country'].str.split('_').str[-1]

In [17]:
df_final['date'] = pd.to_datetime(df_final['date'])

In [18]:
df_daily_check['payer_country'] = df_daily_check['payer']+'_'+df_daily_check['country']

In [19]:
#Traigo 'amount' de la tabla de transacciones diarias 'daily_check'
df_test = pd.merge(df_final, df_daily_check, on=['date','payer_country', 'country'], how='left')

In [20]:
df_test.drop(['payer','tx','promedio x tx'], axis=1, inplace=True)

In [21]:
#Elimino registros donde no tengo amount
df_test = df_test[~df_test['amount'].isnull()]

In [22]:
df_test.to_excel('MAPE_check.xlsx')

### MAPE @ MEAN

In [23]:
statistic = 'mean' # Seleccionar 'mean' o 'p50'

In [24]:
#Calculo el valor absoluto de la diferencia entre 'amount' y 'mean'
df_test['abs_error'] = abs(df_test['amount'] - df_test[statistic])

In [25]:
#Calculo el % de error absoluto para cada fila
df_test['ape'] = df_test['abs_error'] / df_test['amount']

In [27]:
#Calculo el MAPE por payer_country
mape_payer_country = df_test.groupby('payer_country')['ape'].mean()
mape_payer_country

payer_country
24XORO_MEXICO                         0.698201
ABANK (TN)_EL SALVADOR                0.210805
AFEX_CHILE                            0.623288
AFEX_URUGUAY                          0.999183
AFRO INTERNACIONAL_GUINEA             0.769198
                                        ...   
WALMART (UT)_MEXICO                   0.198651
YES BANK_INDIA                        1.428390
ZEEPAY_CAMEROON                       0.791684
ZEEPAY_COTE D'IVOIRE (IVORY COAST)    1.076883
ZEEPAY_GHANA                          0.541112
Name: ape, Length: 129, dtype: float64

In [29]:
df_test.head()

payer_country_encoder       date  p10  p50  p90     mean  \
10                    107 2023-04-11  0.0  0.0  0.0  44.0829   
13                    107 2023-04-14  0.0  0.0  0.0  63.9666   
14                    107 2023-04-15  0.0  0.0  0.0  38.0300   
19                    107 2023-04-20  0.0  0.0  0.0   6.6200   
27                    107 2023-04-28  0.0  0.0  0.0  51.7851   

                 payer_country country   amount  abs_error       ape  
10  TRANSFERTO - THUNES_FRANCE  FRANCE   112.00    67.9171  0.606403  
13  TRANSFERTO - THUNES_FRANCE  FRANCE  1111.11  1047.1434  0.942430  
14  TRANSFERTO - THUNES_FRANCE  FRANCE   100.00    61.9700  0.619700  
19  TRANSFERTO - THUNES_FRANCE  FRANCE   435.00   428.3800  0.984782  
27  TRANSFERTO - THUNES_FRANCE  FRANCE   212.00   160.2149  0.755731

In [59]:
#Calculo el MAPE por country
grouped = df_test.groupby(['date', 'country']).agg({'amount': 'sum', 'mean': 'sum'}).reset_index()

In [61]:
grouped[grouped['country'] == 'BRAZIL'].sort_values('date').head()

date country     amount           mean
3   2023-04-01  BRAZIL  444502.43  569563.011719
35  2023-04-02  BRAZIL  398352.87  539812.953125
68  2023-04-03  BRAZIL  835308.23  861290.867188
99  2023-04-04  BRAZIL  672845.16  722125.476562
133 2023-04-05  BRAZIL  643068.05  557620.585938

In [62]:
grouped['abs_error'] = abs(grouped['amount'] - grouped['mean'])
grouped['ape'] = grouped['abs_error'] / grouped['amount']

In [63]:
grouped.head()

date     country     amount           mean      abs_error       ape
0 2023-04-01   ARGENTINA     680.19     594.680634      85.509366  0.125714
1 2023-04-01  BANGLADESH     985.00    3626.518555    2641.518555  2.681745
2 2023-04-01     BOLIVIA   14416.00   17116.436035    2700.436035  0.187322
3 2023-04-01      BRAZIL  444502.43  569563.011719  125060.581719  0.281350
4 2023-04-01    CAMEROON    2446.08      12.503193    2433.576807  0.994888

In [64]:
grouped_by_country = grouped.groupby('country').agg({'ape': 'sum', 'date': 'count'}).reset_index()

In [65]:
grouped_by_country['mape'] = grouped_by_country['ape']/grouped_by_country['date']

In [66]:
grouped_by_country

country          ape  date       mape
0                        ARGENTINA   113.437414    89   1.274578
1                       BANGLADESH  1467.800983    91  16.129681
2                          BELGIUM    23.148521    23   1.006457
3                            BENIN    19.347023    25   0.773881
4                          BOLIVIA    36.552439   100   0.365524
5                           BRAZIL    20.034276   100   0.200343
6                     BURKINA FASO    44.728265    45   0.993961
7                         CAMEROON    64.649528    80   0.808119
8                            CHILE    49.577441   100   0.495774
9                         COLOMBIA    25.198123   100   0.251981
10                      COSTA RICA    43.753497   100   0.437535
11     COTE D'IVOIRE (IVORY COAST)    57.734220    61   0.946463
12              DOMINICAN REPUBLIC    17.749672   100   0.177497
13                         ECUADOR    18.976289   100   0.189763
14                     EL SALVADOR    10.090209   100   0.100902
15                          FRANCE    17.549894    22   0.797722
16                     GAMBIA, THE   259.554940    93   2.790913
17                         GERMANY    27.531470    30   0.917716
18                           GHANA    54.111192   100   0.541112
19                       GUATEMALA    14.137974   100   0.141380
20                          GUINEA    73.085904    88   0.830522
21                           HAITI    18.489163   100   0.184892
22                        HONDURAS    19.117324   100   0.191173
23                           INDIA   137.125452    96   1.428390
24                       INDONESIA    54.712074   100   0.547121
25                           ITALY    26.694733    31   0.861120
26                         JAMAICA    20.785205    22   0.944782
27                         LIBERIA    26.062327    33   0.789767
28                        MALAYSIA     2.481354     3   0.827118
29                            MALI    45.491489    47   0.967904
30                          MEXICO    12.870905   100   0.128709
31                           NEPAL    79.126848    84   0.941986
32                       NICARAGUA    23.122971   100   0.231230
33                         NIGERIA   103.429571   100   1.034296
34                            PERU    21.010006   100   0.210100
35                     PHILIPPINES    23.452575   100   0.234526
36                        PORTUGAL    76.323595    81   0.942267
37                         SENEGAL    82.991218    83   0.999894
38                    SIERRA LEONE    38.637418   100   0.386374
39                           SPAIN    89.611617   100   0.896116
40                        THAILAND    10.954703    15   0.730314
41                            TOGO    81.807694    75   1.090769
42                  UNITED KINGDOM    22.176315    27   0.821345
43                   UNITED STATES    15.387262   100   0.153873
44  UNITED STATES (ATM & DEPOSITS)    76.498153    99   0.772709
45                         URUGUAY    17.985300    18   0.999183